In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import re
import string
import pandas as pd

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
from textblob import TextBlob

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding,LSTM, SimpleRNN, Dense, Dropout
from transformers import BertTokenizer, TFBertForSequenceClassification

!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/


import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

2025-07-08 09:25:28.249185: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751966728.620111      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751966728.730134      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
file_path = "/kaggle/input/financial-sentiment-analysis/data.csv"
df = pd.read_csv(file_path)
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [4]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_sentence'] = df['Sentence'].apply(clean_text)

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Sentiment'])

In [5]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

df['processed_sentence'] = df['clean_sentence'].apply(preprocess_text)
df.head()

,Sentence,Sentiment,clean_sentence,label,processed_sentence
0,The GeoSolutions technology will leverage Bene...,positive,the geosolutions technology will leverage bene...,2,geosolutions technology leverage benefon gps s...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,esi on lows down to bk a real possibility,0,esi lows bk real possibility
2,"For the last quarter of 2010 , Componenta 's n...",positive,for the last quarter of componenta s net sales...,2,last quarter componenta net sales doubled eurm...
3,According to the Finnish-Russian Chamber of Co...,neutral,according to the finnishrussian chamber of com...,1,according finnishrussian chamber commerce majo...
4,The Swedish buyout firm has sold its remaining...,neutral,the swedish buyout firm has sold its remaining...,1,swedish buyout firm sold remaining percent sta...


In [6]:
nlp = spacy.load("en_core_web_sm")

def advanced_nlp_preprocessing(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space]
    return ' '.join(tokens)

df['nlp_processed_sentence'] = df['clean_sentence'].apply(advanced_nlp_preprocessing)

df.head()

,Sentence,Sentiment,clean_sentence,label,processed_sentence,nlp_processed_sentence
0,The GeoSolutions technology will leverage Bene...,positive,the geosolutions technology will leverage bene...,2,geosolutions technology leverage benefon gps s...,geosolution technology leverage benefon s gps ...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,esi on lows down to bk a real possibility,0,esi lows bk real possibility,esi low bk real possibility
2,"For the last quarter of 2010 , Componenta 's n...",positive,for the last quarter of componenta s net sales...,2,last quarter componenta net sales doubled eurm...,quarter componenta s net sale double eurm eurm...
3,According to the Finnish-Russian Chamber of Co...,neutral,according to the finnishrussian chamber of com...,1,according finnishrussian chamber commerce majo...,accord finnishrussian chamber commerce major c...
4,The Swedish buyout firm has sold its remaining...,neutral,the swedish buyout firm has sold its remaining...,1,swedish buyout firm sold remaining percent sta...,swedish buyout firm sell remain percent stake ...


In [7]:
def textblob_nlp_preprocessing(text):
    words = TextBlob(text).words
    lemmatized_words = [word.lemmatize() for word in words]
    return ' '.join(lemmatized_words)

df['textblob_processed_sentence'] = df['clean_sentence'].apply(textblob_nlp_preprocessing)

df.head()

,Sentence,Sentiment,clean_sentence,label,processed_sentence,nlp_processed_sentence,textblob_processed_sentence
0,The GeoSolutions technology will leverage Bene...,positive,the geosolutions technology will leverage bene...,2,geosolutions technology leverage benefon gps s...,geosolution technology leverage benefon s gps ...,the geosolutions technology will leverage bene...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,esi on lows down to bk a real possibility,0,esi lows bk real possibility,esi low bk real possibility,esi on low down to bk a real possibility
2,"For the last quarter of 2010 , Componenta 's n...",positive,for the last quarter of componenta s net sales...,2,last quarter componenta net sales doubled eurm...,quarter componenta s net sale double eurm eurm...,for the last quarter of componenta s net sale ...
3,According to the Finnish-Russian Chamber of Co...,neutral,according to the finnishrussian chamber of com...,1,according finnishrussian chamber commerce majo...,accord finnishrussian chamber commerce major c...,according to the finnishrussian chamber of com...
4,The Swedish buyout firm has sold its remaining...,neutral,the swedish buyout firm has sold its remaining...,1,swedish buyout firm sold remaining percent sta...,swedish buyout firm sell remain percent stake ...,the swedish buyout firm ha sold it remaining p...


In [8]:
nltk.download('wordnet')

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def stemming_lemmatization(text):
    words = text.split()
    words = [lemmatizer.lemmatize(stemmer.stem(word)) for word in words]
    return ' '.join(words)

df['stem_lemmatized_sentence'] = df['clean_sentence'].apply(stemming_lemmatization)

df.head()

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Sentence,Sentiment,clean_sentence,label,processed_sentence,nlp_processed_sentence,textblob_processed_sentence,stem_lemmatized_sentence
0,The GeoSolutions technology will leverage Bene...,positive,the geosolutions technology will leverage bene...,2,geosolutions technology leverage benefon gps s...,geosolution technology leverage benefon s gps ...,the geosolutions technology will leverage bene...,the geosolut technolog will leverag benefon s ...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,esi on lows down to bk a real possibility,0,esi lows bk real possibility,esi low bk real possibility,esi on low down to bk a real possibility,esi on low down to bk a real possibl
2,"For the last quarter of 2010 , Componenta 's n...",positive,for the last quarter of componenta s net sales...,2,last quarter componenta net sales doubled eurm...,quarter componenta s net sale double eurm eurm...,for the last quarter of componenta s net sale ...,for the last quarter of componenta s net sale ...
3,According to the Finnish-Russian Chamber of Co...,neutral,according to the finnishrussian chamber of com...,1,according finnishrussian chamber commerce majo...,accord finnishrussian chamber commerce major c...,according to the finnishrussian chamber of com...,accord to the finnishrussian chamber of commer...
4,The Swedish buyout firm has sold its remaining...,neutral,the swedish buyout firm has sold its remaining...,1,swedish buyout firm sold remaining percent sta...,swedish buyout firm sell remain percent stake ...,the swedish buyout firm ha sold it remaining p...,the swedish buyout firm ha sold it remain perc...


In [9]:
def custom_text_preprocessing(text):
    contractions = {
        "isn't": "is not", "aren't": "are not", "can't": "cannot", "couldn't": "could not",
        "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
        "hasn't": "has not", "haven't": "have not", "shouldn't": "should not", "wasn't": "was not",
        "weren't": "were not", "won't": "will not", "wouldn't": "would not", "i'm": "i am",
        "it's": "it is", "you're": "you are", "he's": "he is", "she's": "she is", "we're": "we are",
        "they're": "they are", "i've": "i have", "we've": "we have", "they've": "they have"
    }
    
    words = text.split()
    words = [contractions[word] if word in contractions else word for word in words]
    words = [word.strip() for word in words if len(word) > 1]  
    
    return ' '.join(words)

df['custom_nlp_sentence'] = df['clean_sentence'].apply(custom_text_preprocessing)

df.head()

,Sentence,Sentiment,clean_sentence,label,processed_sentence,nlp_processed_sentence,textblob_processed_sentence,stem_lemmatized_sentence,custom_nlp_sentence
0,The GeoSolutions technology will leverage Bene...,positive,the geosolutions technology will leverage bene...,2,geosolutions technology leverage benefon gps s...,geosolution technology leverage benefon s gps ...,the geosolutions technology will leverage bene...,the geosolut technolog will leverag benefon s ...,the geosolutions technology will leverage bene...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,esi on lows down to bk a real possibility,0,esi lows bk real possibility,esi low bk real possibility,esi on low down to bk a real possibility,esi on low down to bk a real possibl,esi on lows down to bk real possibility
2,"For the last quarter of 2010 , Componenta 's n...",positive,for the last quarter of componenta s net sales...,2,last quarter componenta net sales doubled eurm...,quarter componenta s net sale double eurm eurm...,for the last quarter of componenta s net sale ...,for the last quarter of componenta s net sale ...,for the last quarter of componenta net sales d...
3,According to the Finnish-Russian Chamber of Co...,neutral,according to the finnishrussian chamber of com...,1,according finnishrussian chamber commerce majo...,accord finnishrussian chamber commerce major c...,according to the finnishrussian chamber of com...,accord to the finnishrussian chamber of commer...,according to the finnishrussian chamber of com...
4,The Swedish buyout firm has sold its remaining...,neutral,the swedish buyout firm has sold its remaining...,1,swedish buyout firm sold remaining percent sta...,swedish buyout firm sell remain percent stake ...,the swedish buyout firm ha sold it remaining p...,the swedish buyout firm ha sold it remain perc...,the swedish buyout firm has sold its remaining...


In [10]:
def remove_repeated_characters(text):
    return re.sub(r'(.)\1+', r'\1\1', text)

df['no_repeated_chars_sentence'] = df['custom_nlp_sentence'].apply(remove_repeated_characters)

df.head()

,Sentence,Sentiment,clean_sentence,label,processed_sentence,nlp_processed_sentence,textblob_processed_sentence,stem_lemmatized_sentence,custom_nlp_sentence,no_repeated_chars_sentence
0,The GeoSolutions technology will leverage Bene...,positive,the geosolutions technology will leverage bene...,2,geosolutions technology leverage benefon gps s...,geosolution technology leverage benefon s gps ...,the geosolutions technology will leverage bene...,the geosolut technolog will leverag benefon s ...,the geosolutions technology will leverage bene...,the geosolutions technology will leverage bene...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,esi on lows down to bk a real possibility,0,esi lows bk real possibility,esi low bk real possibility,esi on low down to bk a real possibility,esi on low down to bk a real possibl,esi on lows down to bk real possibility,esi on lows down to bk real possibility
2,"For the last quarter of 2010 , Componenta 's n...",positive,for the last quarter of componenta s net sales...,2,last quarter componenta net sales doubled eurm...,quarter componenta s net sale double eurm eurm...,for the last quarter of componenta s net sale ...,for the last quarter of componenta s net sale ...,for the last quarter of componenta net sales d...,for the last quarter of componenta net sales d...
3,According to the Finnish-Russian Chamber of Co...,neutral,according to the finnishrussian chamber of com...,1,according finnishrussian chamber commerce majo...,accord finnishrussian chamber commerce major c...,according to the finnishrussian chamber of com...,accord to the finnishrussian chamber of commer...,according to the finnishrussian chamber of com...,according to the finnishrussian chamber of com...
4,The Swedish buyout firm has sold its remaining...,neutral,the swedish buyout firm has sold its remaining...,1,swedish buyout firm sold remaining percent sta...,swedish buyout firm sell remain percent stake ...,the swedish buyout firm ha sold it remaining p...,the swedish buyout firm ha sold it remain perc...,the swedish buyout firm has sold its remaining...,the swedish buyout firm has sold its remaining...


In [11]:
def remove_short_words(text, min_length=3):
    words = text.split()
    filtered_words = [word for word in words if len(word) >= min_length]
    return ' '.join(filtered_words)

df['filtered_short_words_sentence'] = df['no_repeated_chars_sentence'].apply(lambda x: remove_short_words(x, min_length=3))

df.head()

,Sentence,Sentiment,clean_sentence,label,processed_sentence,nlp_processed_sentence,textblob_processed_sentence,stem_lemmatized_sentence,custom_nlp_sentence,no_repeated_chars_sentence,filtered_short_words_sentence
0,The GeoSolutions technology will leverage Bene...,positive,the geosolutions technology will leverage bene...,2,geosolutions technology leverage benefon gps s...,geosolution technology leverage benefon s gps ...,the geosolutions technology will leverage bene...,the geosolut technolog will leverag benefon s ...,the geosolutions technology will leverage bene...,the geosolutions technology will leverage bene...,the geosolutions technology will leverage bene...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,esi on lows down to bk a real possibility,0,esi lows bk real possibility,esi low bk real possibility,esi on low down to bk a real possibility,esi on low down to bk a real possibl,esi on lows down to bk real possibility,esi on lows down to bk real possibility,esi lows down real possibility
2,"For the last quarter of 2010 , Componenta 's n...",positive,for the last quarter of componenta s net sales...,2,last quarter componenta net sales doubled eurm...,quarter componenta s net sale double eurm eurm...,for the last quarter of componenta s net sale ...,for the last quarter of componenta s net sale ...,for the last quarter of componenta net sales d...,for the last quarter of componenta net sales d...,for the last quarter componenta net sales doub...
3,According to the Finnish-Russian Chamber of Co...,neutral,according to the finnishrussian chamber of com...,1,according finnishrussian chamber commerce majo...,accord finnishrussian chamber commerce major c...,according to the finnishrussian chamber of com...,accord to the finnishrussian chamber of commer...,according to the finnishrussian chamber of com...,according to the finnishrussian chamber of com...,according the finnishrussian chamber commerce ...
4,The Swedish buyout firm has sold its remaining...,neutral,the swedish buyout firm has sold its remaining...,1,swedish buyout firm sold remaining percent sta...,swedish buyout firm sell remain percent stake ...,the swedish buyout firm ha sold it remaining p...,the swedish buyout firm ha sold it remain perc...,the swedish buyout firm has sold its remaining...,the swedish buyout firm has sold its remaining...,the swedish buyout firm has sold its remaining...


In [12]:
def remove_urls(text):
    return re.sub(r'http[s]?://\S+', '', text)

df['no_urls_sentence'] = df['filtered_short_words_sentence'].apply(remove_urls)
df.head()

,Sentence,Sentiment,clean_sentence,label,processed_sentence,nlp_processed_sentence,textblob_processed_sentence,stem_lemmatized_sentence,custom_nlp_sentence,no_repeated_chars_sentence,filtered_short_words_sentence,no_urls_sentence
0,The GeoSolutions technology will leverage Bene...,positive,the geosolutions technology will leverage bene...,2,geosolutions technology leverage benefon gps s...,geosolution technology leverage benefon s gps ...,the geosolutions technology will leverage bene...,the geosolut technolog will leverag benefon s ...,the geosolutions technology will leverage bene...,the geosolutions technology will leverage bene...,the geosolutions technology will leverage bene...,the geosolutions technology will leverage bene...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,esi on lows down to bk a real possibility,0,esi lows bk real possibility,esi low bk real possibility,esi on low down to bk a real possibility,esi on low down to bk a real possibl,esi on lows down to bk real possibility,esi on lows down to bk real possibility,esi lows down real possibility,esi lows down real possibility
2,"For the last quarter of 2010 , Componenta 's n...",positive,for the last quarter of componenta s net sales...,2,last quarter componenta net sales doubled eurm...,quarter componenta s net sale double eurm eurm...,for the last quarter of componenta s net sale ...,for the last quarter of componenta s net sale ...,for the last quarter of componenta net sales d...,for the last quarter of componenta net sales d...,for the last quarter componenta net sales doub...,for the last quarter componenta net sales doub...
3,According to the Finnish-Russian Chamber of Co...,neutral,according to the finnishrussian chamber of com...,1,according finnishrussian chamber commerce majo...,accord finnishrussian chamber commerce major c...,according to the finnishrussian chamber of com...,accord to the finnishrussian chamber of commer...,according to the finnishrussian chamber of com...,according to the finnishrussian chamber of com...,according the finnishrussian chamber commerce ...,according the finnishrussian chamber commerce ...
4,The Swedish buyout firm has sold its remaining...,neutral,the swedish buyout firm has sold its remaining...,1,swedish buyout firm sold remaining percent sta...,swedish buyout firm sell remain percent stake ...,the swedish buyout firm ha sold it remaining p...,the swedish buyout firm ha sold it remain perc...,the swedish buyout firm has sold its remaining...,the swedish buyout firm has sold its remaining...,the swedish buyout firm has sold its remaining...,the swedish buyout firm has sold its remaining...


In [13]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['no_urls_sentence'])

sequences = tokenizer.texts_to_sequences(df['no_urls_sentence'])
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(df['Sentiment'])

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels_encoded, test_size=0.2, random_state=42)

rnn_model = Sequential([
    Embedding(10000, 128, input_length=100),
    SimpleRNN(128, return_sequences=True),
    Dropout(0.5),
    SimpleRNN(64),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

rnn_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

rnn_model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-07-08 09:31:27.393062: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


147/147 ━━━━━━━━━━━━━━━━━━━━ 16s 65ms/step - accuracy: 0.4890 - loss: 1.0384 - val_accuracy: 0.5321 - val_loss: 0.9982
Epoch 2/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 9s 64ms/step - accuracy: 0.5149 - loss: 1.0049 - val_accuracy: 0.5321 - val_loss: 0.9859
Epoch 3/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 9s 60ms/step - accuracy: 0.5414 - loss: 0.9884 - val_accuracy: 0.5321 - val_loss: 0.9729
Epoch 4/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 9s 60ms/step - accuracy: 0.5169 - loss: 1.0043 - val_accuracy: 0.5321 - val_loss: 0.9849
Epoch 5/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 9s 60ms/step - accuracy: 0.5440 - loss: 0.9846 - val_accuracy: 0.5329 - val_loss: 0.9866


In [14]:
lstm_model = Sequential([
    Embedding(10000, 128, input_length=100),
    LSTM(128, return_sequences=True),
    Dropout(0.5),
    LSTM(64),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

lstm_model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 31s 167ms/step - accuracy: 0.5273 - loss: 0.9961 - val_accuracy: 0.5321 - val_loss: 0.9854
Epoch 2/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 24s 163ms/step - accuracy: 0.5298 - loss: 0.9948 - val_accuracy: 0.5321 - val_loss: 0.9877
Epoch 3/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 23s 158ms/step - accuracy: 0.5463 - loss: 0.9799 - val_accuracy: 0.5321 - val_loss: 0.9850
Epoch 4/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 24s 161ms/step - accuracy: 0.5420 - loss: 0.9726 - val_accuracy: 0.5321 - val_loss: 0.9845
Epoch 5/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 24s 161ms/step - accuracy: 0.5362 - loss: 0.9736 - val_accuracy: 0.5321 - val_loss: 0.9852
